# REDES NEURAIS - RELATÓRIO PROJETO 1


## Grupo
- Eduardo Souza - 9293481
- Edresson Casanova - 11572715
- Marcela Prince Antunes - 11548673


## Objetivo

O objetivo deste projeto é desenvolver uma arquitetura de rede neural MLP para classificação e aproximação, testá-la com diferentes parâmetros e comparar os resultados. Como entrada, serão usados conjuntos de dados sobre vinhos e músicas.

## Módulos Externos e Funções Auxiliares

In [0]:
import numpy as np
from scipy.special import expit
import random
from tqdm import tqdm

#Função logistica, também conhecida como função sigmoidal
def logistic(x, der=False):
	return expit(x) if not der else expit(x) * (1-expit(x))

#Função tangente hiperbolica
def tanh(x, der=False):
	return np.tanh(x)  if not der else  1 - np.tanh(x)**2

#Função ReLU(rectified linear unit)
def ReLU(x, der=False):
	return  np.max(0, x) if not der else (1 if (x>0) else 0)

#Função softplus(suavização da ReLU)
def softplus(x, der=False):
	return  np.log(1 + np.exp(x)) if not der else 1/(1 + np.exp(-x))

#Função linear f(x)=x
def linear(x, der=False):
	return x if not der else 1

#Função de Loss de erro quadrático
def squared_error(pred, true, der=False):
	return np.sum(np.square(pred-true)) if not der else 2*(pred-true)

#Calcula o erro quadrático médio entre a saida predita e a real
def mean_squared_error(pred, true):
  return np.sum(np.square(pred-true))/len(true)

#Calcula a acuracia dados a saida predita e a real
def accuracy(pred, true):		
	return np.count_nonzero(np.argmax(pred, axis=1) == np.argmax(true, axis=1))/len(pred)

## Definição da MLP

A classe MLP implementa todos os métodos necessários para treinar um Perceptron multi camadas e executar avaliações utilizando os pesos obtidos. A seguir os principais métodos serão detalhados.

In [0]:
class MLP:
	#Gera a MLP
	def __init__(self, input_size, layers_sizes, actv_funcs=None, loss_func=squared_error, metrics=[("Accuracy", accuracy)]):
		#Define todas as funções de ativação como sendo a logistica caso não definidas
		if not actv_funcs:
			actv_funcs = [logistic for _ in range(len(layers_sizes))]

		#Verificação da entrada
		assert input_size > 0
		assert all([layer_size > 0 for layer_size in layers_sizes])
		assert len(layers_sizes) == len(actv_funcs)

		#Atribuição das funções e métricas
		self.set_actv_funcs(actv_funcs)
		self.set_loss_func(loss_func)
		self.set_metrics(metrics)

		#Gera as camadas com pesos entre -1 e 1
		self.layers = [np.random.random_sample((layers_sizes[0], input_size+1))*2.0 - 1]
		self.layers += [np.random.random_sample((layers_sizes[i], layers_sizes[i-1]+1))*2.0 - 1 for i in range(1, len(layers_sizes))]
	
		#Vetores auxiliares
		self.last_inputs = [np.ones((input_size+1, ))] + [np.ones((layer_size+1, )) for layer_size in layers_sizes]
		self.last_nets = [np.empty((layer_size, )) for layer_size in layers_sizes]
		self.last_ders = [np.zeros(layer.shape) for layer in self.layers]

	#Atribuição das funções de ativação
	def set_actv_funcs(self, actv_funcs):
		#Verificação da entrada e atribuição
		assert all([callable(actv_func) for actv_func in actv_funcs])
		self.actv_funcs = actv_funcs

	#Atribuição da função de loss
	def set_loss_func(self, loss_func):
		#Verificação da entrada e atribuição
		assert callable(loss_func)
		self.loss_func = loss_func

	#Atribuição das métricas
	def set_metrics(self, metrics):
		#Verificação da entrada e atribuição
		assert all([callable(metric[1]) for metric in metrics])
		self.metrics = metrics

	#Faz o forward da MLP para uma sample
	def _predict_single(self, x):
		#Executar o forward camada por camada
		self.last_inputs[0][:-1] = x
		for i in range(len(self.layers)):
			self.last_nets[i] = np.sum(self.last_inputs[i] * self.layers[i], axis=1)
			self.last_inputs[i+1][:-1] = self.actv_funcs[i](self.last_nets[i], False)

		#Retorna a saida da ultima camada
		return np.copy(self.last_inputs[-1][:-1])

	#Faz o forward da MLP para todo o dataset de entrada
	def predict(self, x):
		#Verificação da entrada
		assert type(x) == np.ndarray
		#Caso seja apenas uma sample, executa predict_single
		if (len(x.shape) == 1):
			assert len(x) == self.layers[0].shape[1]-1
			return _predict_single(x)
		assert len(x.shape) == 2
		assert x.shape[1] == self.layers[0].shape[1]-1

		#Executa predict_single para cada sample, e retorna as saidas em uma matriz
		return np.stack([self._predict_single(sample) for sample in x], axis=0)

	#Calcula a loss média para todo o dataset de entrada
	def _get_loss(self, pred, true):
		return np.sum([self.loss_func(sample_pred, sample_true, False) for sample_pred, sample_true in zip(pred, true)]) / len(true)

	#Calcula as métricas para todo o dataset de entrada
	def _get_metrics(self, pred, true):
		return [(metric[0], metric[1](pred, true)) for metric in self.metrics]

	def evaluate(self, x, y):
		#Verificação da entrada
		assert (type(x) == np.ndarray) and (type(y) == np.ndarray)
		assert (len(x.shape) == 2) and (len(y.shape) == 2)
		assert len(x) == len(y)

		#Calculo das métricas e da loss
		pred = self.predict(x)
		return [("Loss", self._get_loss(pred, y))] + self._get_metrics(pred, y)


	#Faz o backpropagation da MLP
	def fit(self, x, y, epochs=1, loss_thresh=0.001, learning_rate=0.1, momentum=0.01, reset_momentum=False, return_history=False, verbose=False):
		#Verificação da entrada
		assert (type(x) == np.ndarray) and (type(y) == np.ndarray)
		assert (len(x.shape) == 2) and (len(y.shape) == 2)
		assert len(x) == len(y)
		assert epochs > 0
		assert loss_thresh >= 0
		assert learning_rate > 0
		assert (momentum >= 0) and (momentum < 1)
		
		#Reseta o momentum armazenado se necessário
		self.last_ders = [np.zeros(layer.shape) for layer in self.layers] if reset_momentum else self.last_ders

		#Faz o loop do fit até o numero máximo de epochs ou até a loss ser menor que o threshold
		cur_epoch = 0
		cur_epoch_loss = loss_thresh+1
		history = []
		while(cur_epoch_loss > loss_thresh and cur_epoch < epochs):
			#Fitar sample a sample
			for sample, ground_truth in zip(x, y):
				#------Derivar e propagar para todas as camada------
				#---Derivação inicialmente da última camada, fora do loop por ser diferente das outras---
				#A variável der_backprop armazena a derivada que esta sendo propagada para trás
				der_backprop = self.loss_func(self._predict_single(sample), ground_truth, True) * self.actv_funcs[-1](self.last_nets[-1], True)
				#Necessrio para multiplicar na linha de baixo, apenas altera o formato do array
				der_backprop = np.expand_dims(der_backprop, axis=1)
				#A variável cur_der armazena a derivada para os pesos da camada atual na iteração atual, antes do momentum
				cur_der = der_backprop * np.tile(self.last_inputs[-2], (len(self.layers[-1]), 1))
				#A variável self.last_ders armazena o estado atual do momentum da derivada para cada camada
				self.last_ders[-1] = momentum * self.last_ders[-1] + learning_rate * cur_der
				#---Derivação em loop para as camadas escondidas, da camada mais proxima a saída à mais próxima a entrada---
				for i in range(len(self.layers)-2, -1, -1):
					der_backprop = np.sum(der_backprop * self.layers[i+1][:, :-1], axis=0) * self.actv_funcs[i](self.last_nets[i], True)
					der_backprop = np.expand_dims(der_backprop, axis=1)
					cur_der = der_backprop * np.tile(self.last_inputs[i], (len(self.layers[i]), 1))
					self.last_ders[i] = momentum * self.last_ders[i] + learning_rate * cur_der

				#Atualização de todos os pesos de todas as camadas pelo gradiente considerendo momentum
				self.layers = [layer - last_der for layer, last_der in zip(self.layers, self.last_ders)]

			#Pegar a loss média da epoch
			cur_dataset_pred = self.predict(x)
			cur_epoch_loss = self._get_loss(cur_dataset_pred, y)
			#Calcular as métricas e salvar no histórico se necessário
			if return_history or verbose:
				#Calcular as métricas
				cur_metrics = [("Loss", cur_epoch_loss)] + self._get_metrics(cur_dataset_pred, y)
				
				#Printar na tela se verboso
				if verbose:
					print("----------------------------")
					print("\tEPOCH {}".format(cur_epoch))
					[print(metric[0], ": ", metric[1], sep='') for metric in cur_metrics]

				#Salvar no histórico se necessário
				history += cur_metrics if return_history else []

			#Incrementar contador de epoch
			cur_epoch += 1

		return history if return_history else None

### Construtor

A implementação é bastante genérica e permite criar redes MLP com diferentes arquiteturas e utilizando diferentes funções de ativação e perda. A partir dos argumentos **input_size** e **layer_sizes** são criados os vetores que armazenarão os pesos de cada neurônio. Outros argumentos importantes são o **actv_funcs**, que define as funções de ativação para cada camada, e **loss_func**, que define a função de perda.

### _predict_single

Executa a avaliação de uma entrada, gerando uma saída. Para cada camada, calcula o somatório utilizando a saída da camada anterior (ou a própria entrada, no caso da primeira camada) e os pesos da camada atual. O resultado do somatório é, então, passado para a função de ativação da camada atual, gerando a saída, que será entrada para a próxima camada. A saída da última camada é retornado como resultado da função.

### predict

Recebe um conjunto de testes e, para cada item do conjunto, executa o método **_predict_single**. Retorna o resultado em forma de matriz, sendo um linha para cada item do conjunto de testes.

### evaluate

Recebe um conjunto de testes e um conjunto de resultados esperados. O conjunto de testes (**x**) é passado para o método **predict**, gerando uma matriz com todos os resultados (**pred**). Posteriormente, os resultados calculados e os resultados esperados (**y**) são passados para a função **_get_loss**, que calcula a perda, utilizando a função de perda, para cada par.

### fit

Método que implementa o treinamento. Assim como o construtor, também é bastante genérico, permitindo definir via argumentos vários parâmetros do treinamento, como quantidade de épocas, limite de perda, taxa de treinamento e *momentum*.

O treinamento consiste em um *loop*, cujo teste de parada é o número de épocas e o limite da perda. Para cada época, um segundo *loop* executa o mecanismo *backpropagation* e a atualização dos pesos para cada item do conjunto de treinamento. Ao final de cada época, um novo valor de perda é calculado, que, por sua vez, será usado para verificar se houve ou não convergência.

## Download dos Datasets


In [3]:
!rm default_features_1059_tracks.txt
!rm  winequality-red.csv
!wget https://www.dropbox.com/s/6wt53zsk8r9co0u/default_features_1059_tracks.txt?dl=0 -O default_features_1059_tracks.txt
!wget https://www.dropbox.com/s/yi652oiw0g1q01a/winequality-red.csv?dl=0 -O winequality-red.csv
!ls

rm: cannot remove 'winequality-red.csv': No such file or directory
--2019-09-09 21:04:15--  https://www.dropbox.com/s/6wt53zsk8r9co0u/default_features_1059_tracks.txt?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/6wt53zsk8r9co0u/default_features_1059_tracks.txt [following]
--2019-09-09 21:04:16--  https://www.dropbox.com/s/raw/6wt53zsk8r9co0u/default_features_1059_tracks.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc61804c5486f29e768593e0f561.dl.dropboxusercontent.com/cd/0/inline/AoST25QXGV0bTXE_HQ2DynCjXGoxaRQ4HRULKZdy3Eny0_pRBVkbgv2D2BPyLBHygopWAiCk2Ap058ZEEmY-XJCVsVQVc_RauxSBWAl5UJevPcQQ18gtfHz4RNaSFMfNQWY/file# [following]
--2019-09-09 21:04:16--  https://uc61804c5486f29e768593e0f561.dl.dropboxuserconten

## Dataset 1: winequality-red.csv (classificação)


### Preprocessando Dataset 1

O conjunto 1 consiste em uma tabela com 13 colunas. A primeira coluna possui um index e é ignorada durante o treinamento e avaliação. A colune 13 possui a classificação do vinho e será usada como referência durante o treinamento, como saída esperada. As colunas de 2 a 12 possuem valores numéricos indicando diferentes características de cada um dos vinhos do conjunto.

O processo de normalização utiliza a função abaixo:

$x_{i}^{'} = \frac{x_i - x_{min}}{x_{max}-x_{min}}$

Após a normalização, o conjunto de dados é reorganizado de forma aleatória.

In [0]:
X = []
Y = []
list_map = {"mid":[1,0,0],"good":[0,1,0],"bad":[0,0,1]}

with open('winequality-red.csv') as f:
  lines = f.readlines()
  for i in range(len(lines)):
    #ignorar a primeira linha
    if i ==0:
      continue
    splits = lines[i].split(',')
    x_i = splits[1:len(splits)-1]# de 1 a penultima posição pois a primeira posição é o numero da amostra uma feature que pode ser removida.
    y_i = splits[-1].lower().replace(' ','').replace('\n', '') # ultima posição onde está o y, replace os espaços e deixar tudo em minusculo para usar o map
    x_i = list( map(float, x_i) ) # convert todos os elementos de x_i para float
    X.append(x_i)
    Y.append(list_map[y_i])
    

# normalizando entre 0 e 1 usando o min max
X = (X - np.min(X, axis=0))/np.ptp(X, axis=0) 

XY = list(zip(X,Y))

random.shuffle(XY)

X, Y = zip(*XY)

del XY

X = np.array(X)
Y = np.array(Y)

print("Total de Amostras:", X.shape[0], " Número de Features: ", X.shape[1])
    


Total de Amostras: 1599  Número de Features:  11


### Criando o modelo neural para o problema de  classificação

Para avaliar o comportamento da MLP foram executados treinamentos e avaliações utilizando-se diferentes topologias, parâmetros de treinamento e tamanhos dos conjuntos de treinamento e teste. 

O trecho de código abaixo define alguns diferentes valores para os seguintes parâmetros: topologia (quantidade de camadas e neurônios por camada, exemplo [5,3] indica que a rede tem 1 camada escondida com 5 neuronios e a camada de saída possui 3 neurônios), taxa de aprendizagem, *momentum*, quatidade de épocas e percentual do conjunto de dados utilizado nos testes.

Para cada combinação destes parâmetros uma MPL é criada, treinada e avaliada. A função de ativação usada é a logística, para todas as camadas.

In [0]:
#definindo listas para descida coordenada de encosta
topologies_list = [[5,3],[10,8,3],[11,8,3]] # em cada layer cada item da lista indica 1 nova camada e o numero de neuronios é igual ao numero de cada posição ex: [7,4,3] cria 3 camadas sendo a primeira com 7 neuronios, a segunda com 4 e a ultima com 3. 
learning_rates_list= [0.00001,0.01,0.9] 
momentums_list= [0.00001,0.001,0.9]
epochs_list=[50, 300, 500]
test_percentages = [10,30,70]# porcentagem de amostras de teste

results = []
results_history =[]
for l in tqdm(range(len(topologies_list))):
  for lr in learning_rates_list:
    for mo in momentums_list:
      for ep in epochs_list:
        for percent in test_percentages:
          lenx = len(X)
          skip = int(lenx * (float(percent) / 100))
          Xtrain = X[0:lenx-skip]
          Ytrain = Y[0:lenx-skip]
          Xtest = X[lenx-skip:]
          Ytest = Y[lenx-skip:]
          
          layer = topologies_list[l]
          mlp = MLP(input_size=11, layers_sizes=layer,  actv_funcs=None, loss_func=squared_error, metrics=[("Accuracy", accuracy)])#create MLP
          history = mlp.fit(Xtrain, Ytrain, ep, loss_thresh=0.0001, learning_rate=lr, momentum=mo, reset_momentum=True, return_history=True, verbose=False)# treinando
          evaluate = mlp.evaluate(Xtest, Ytest)#predizer conjunto de teste
          results.append([layer,percent,lr,mo,ep,evaluate[0][1],evaluate[1][1]])#salvando resultados
          results_history.append(history)#salvando historico do treinamento para cada experimento

100%|██████████| 3/3 [2:17:41<00:00, 2607.52s/it]


### Visualização dos Resultados

No trecho de código abaixo os resultados gerados anteriormente são apresentados em forma de tabela. Cada linha representa uma combinação de parâmetros e seus respectivos resultados, ou seja, perda e acurácia.

In [0]:
from tabulate import tabulate
newresults = sorted(results, key=lambda k: k[-1]) 
tabela =tabulate(newresults, headers=[ 'Topologia',"Porcentagem de Teste",'Learning_Rate', "Momentum", "Epocas","Loss Média","Acurácia"])
with open('resultados-dataset1-classificacao-ordenado.txt', 'w') as outputfile:
    outputfile.write(tabela)
    
print(tabela)

#### Resultados ordenados pela menor acurácia

|Topologia |Porcentagem de Teste|Learning_Rate|Momentum|Epocas|Loss Média          |Acurácia            |
|----------|--------------------|-------------|--------|------|--------------------|--------------------|
|[11, 8, 3]|70                  |1e-05        |1e-05   |50    |1.28752             |0.0402145           |
|[11, 8, 3]|30                  |1e-05        |1e-05   |50    |1.32926             |0.0438413           |
|[10, 8, 3]|10                  |1e-05        |1e-05   |50    |0.949002            |0.044025199999999994|
|[5, 3]    |30                  |1e-05        |0.001   |50    |0.989267            |0.129436            |
|[5, 3]    |30                  |1e-05        |0.001   |300   |0.779929            |0.129436            |
|[11, 8, 3]|30                  |1e-05        |0.001   |50    |1.20781             |0.129436            |
|[11, 8, 3]|10                  |1e-05        |1e-05   |50    |0.8842049999999999  |0.13836500000000002 |
|[5, 3]    |70                  |1e-05        |1e-05   |50    |1.00803             |0.142985            |
|[10, 8, 3]|70                  |1e-05        |1e-05   |50    |1.35766             |0.142985            |
|[5, 3]    |30                  |1e-05        |1e-05   |50    |0.6575850000000001  |0.14405             |
|[5, 3]    |10                  |1e-05        |0.001   |50    |0.689706            |0.238994            |
|[11, 8, 3]|30                  |0.9          |0.001   |300   |0.309319            |0.770355            |
|[11, 8, 3]|30                  |0.9          |1e-05   |300   |0.29681199999999996 |0.7849689999999999  |
|[10, 8, 3]|30                  |0.9          |1e-05   |500   |0.289088            |0.7870560000000001  |
|[10, 8, 3]|70                  |0.9          |0.001   |500   |0.33826300000000004 |0.7908850000000001  |
|[5, 3]    |70                  |0.9          |1e-05   |300   |0.327778            |0.791778            |
|[5, 3]    |30                  |0.9          |1e-05   |50    |0.274325            |0.793319            |
|[11, 8, 3]|70                  |0.9          |0.001   |300   |0.314645            |0.794459            |
|[5, 3]    |30                  |0.9          |0.001   |50    |0.27432199999999995 |0.797495            |
|[5, 3]    |30                  |0.9          |1e-05   |300   |0.281362            |0.80167             |
|[5, 3]    |30                  |0.9          |1e-05   |500   |0.27271799999999996 |0.805846            |
|[5, 3]    |30                  |0.9          |0.001   |500   |0.295165            |0.805846            |
|[11, 8, 3]|30                  |0.01         |0.9     |300   |0.266045            |0.805846            |
|[11, 8, 3]|70                  |0.9          |1e-05   |300   |0.305028            |0.806971            |
|[10, 8, 3]|70                  |0.9          |1e-05   |300   |0.31860700000000003 |0.807864            |
|[11, 8, 3]|70                  |0.9          |0.001   |500   |0.323806            |0.808758            |
|[10, 8, 3]|70                  |0.9          |0.001   |300   |0.304026            |0.809651            |
|[10, 8, 3]|30                  |0.01         |0.9     |300   |0.25532699999999997 |0.810021            |
|[11, 8, 3]|30                  |0.01         |0.9     |500   |0.259708            |0.810021            |
|[5, 3]    |10                  |0.9          |1e-05   |300   |0.24174600000000002 |0.811321            |
|[5, 3]    |10                  |0.9          |0.001   |300   |0.264888            |0.811321            |
|[10, 8, 3]|70                  |0.01         |0.9     |50    |0.275513            |0.811439            |
|[11, 8, 3]|30                  |0.9          |1e-05   |500   |0.295542            |0.812109            |
|[11, 8, 3]|70                  |0.01         |0.9     |300   |0.28694400000000003 |0.81412             |
|[5, 3]    |70                  |0.9          |0.001   |500   |0.301933            |0.815013            |
|[11, 8, 3]|70                  |0.01         |0.9     |500   |0.297212            |0.815013            |
|[11, 8, 3]|70                  |0.9          |1e-05   |500   |0.310683            |0.815013            |
|[5, 3]    |70                  |0.01         |0.9     |50    |0.26652800000000004 |0.8159069999999999  |
|[5, 3]    |70                  |0.9          |1e-05   |500   |0.30361             |0.8159069999999999  |
|[10, 8, 3]|30                  |0.9          |1e-05   |300   |0.293015            |0.816284            |
|[5, 3]    |70                  |1e-05        |1e-05   |300   |0.47856099999999996 |0.816801            |
|[5, 3]    |70                  |1e-05        |1e-05   |500   |0.49213             |0.816801            |
|[5, 3]    |70                  |1e-05        |0.001   |50    |0.485806            |0.816801            |
|[5, 3]    |70                  |1e-05        |0.001   |300   |0.477317            |0.816801            |
|[5, 3]    |70                  |1e-05        |0.001   |500   |0.5276310000000001  |0.816801            |
|[5, 3]    |70                  |1e-05        |0.9     |50    |0.360004            |0.816801            |
|[5, 3]    |70                  |1e-05        |0.9     |300   |0.318268            |0.816801            |
|[5, 3]    |70                  |1e-05        |0.9     |500   |0.31150300000000003 |0.816801            |
|[5, 3]    |70                  |0.01         |1e-05   |50    |0.30688699999999997 |0.816801            |
|[5, 3]    |70                  |0.01         |0.001   |50    |0.303897            |0.816801            |
|[5, 3]    |70                  |0.9          |0.9     |50    |0.366215            |0.816801            |
|[5, 3]    |70                  |0.9          |0.9     |300   |0.366398            |0.816801            |
|[5, 3]    |70                  |0.9          |0.9     |500   |0.366398            |0.816801            |
|[10, 8, 3]|70                  |1e-05        |1e-05   |300   |0.60845             |0.816801            |
|[10, 8, 3]|70                  |1e-05        |1e-05   |500   |0.44375299999999995 |0.816801            |
|[10, 8, 3]|70                  |1e-05        |0.001   |50    |0.6126020000000001  |0.816801            |
|[10, 8, 3]|70                  |1e-05        |0.001   |300   |0.424903            |0.816801            |
|[10, 8, 3]|70                  |1e-05        |0.001   |500   |0.396918            |0.816801            |
|[10, 8, 3]|70                  |1e-05        |0.9     |50    |0.33169499999999996 |0.816801            |
|[10, 8, 3]|70                  |1e-05        |0.9     |300   |0.312227            |0.816801            |
|[10, 8, 3]|70                  |1e-05        |0.9     |500   |0.314409            |0.816801            |
|[10, 8, 3]|70                  |0.01         |1e-05   |50    |0.309791            |0.816801            |
|[10, 8, 3]|70                  |0.01         |0.001   |50    |0.3096              |0.816801            |
|[10, 8, 3]|70                  |0.9          |0.9     |50    |0.365324            |0.816801            |
|[10, 8, 3]|70                  |0.9          |0.9     |300   |0.366398            |0.816801            |
|[10, 8, 3]|70                  |0.9          |0.9     |500   |0.366399            |0.816801            |
|[11, 8, 3]|70                  |1e-05        |1e-05   |300   |0.335924            |0.816801            |
|[11, 8, 3]|70                  |1e-05        |1e-05   |500   |0.34622800000000004 |0.816801            |
|[11, 8, 3]|70                  |1e-05        |0.001   |50    |0.676238            |0.816801            |
|[11, 8, 3]|70                  |1e-05        |0.001   |300   |0.403163            |0.816801            |
|[11, 8, 3]|70                  |1e-05        |0.001   |500   |0.329359            |0.816801            |
|[11, 8, 3]|70                  |1e-05        |0.9     |50    |0.403854            |0.816801            |
|[11, 8, 3]|70                  |1e-05        |0.9     |300   |0.314078            |0.816801            |
|[11, 8, 3]|70                  |1e-05        |0.9     |500   |0.31340100000000004 |0.816801            |
|[11, 8, 3]|70                  |0.01         |1e-05   |50    |0.30877             |0.816801            |
|[11, 8, 3]|70                  |0.01         |0.001   |50    |0.311422            |0.816801            |
|[11, 8, 3]|70                  |0.9          |0.9     |50    |0.365363            |0.816801            |
|[11, 8, 3]|70                  |0.9          |0.9     |300   |0.366381            |0.816801            |
|[11, 8, 3]|70                  |0.9          |0.9     |500   |0.365055            |0.816801            |
|[5, 3]    |10                  |1e-05        |1e-05   |50    |0.5896020000000001  |0.8176100000000001  |
|[5, 3]    |10                  |1e-05        |1e-05   |300   |0.38758000000000004 |0.8176100000000001  |
|[5, 3]    |10                  |1e-05        |1e-05   |500   |0.327937            |0.8176100000000001  |
|[5, 3]    |10                  |1e-05        |0.001   |300   |0.365735            |0.8176100000000001  |
|[5, 3]    |10                  |1e-05        |0.001   |500   |0.339181            |0.8176100000000001  |
|[5, 3]    |10                  |1e-05        |0.9     |50    |0.328266            |0.8176100000000001  |
|[5, 3]    |10                  |1e-05        |0.9     |300   |0.314661            |0.8176100000000001  |
|[5, 3]    |10                  |1e-05        |0.9     |500   |0.30557399999999996 |0.8176100000000001  |
|[5, 3]    |10                  |0.01         |1e-05   |50    |0.270379            |0.8176100000000001  |
|[5, 3]    |10                  |0.01         |0.001   |50    |0.283877            |0.8176100000000001  |
|[5, 3]    |10                  |0.9          |0.9     |50    |0.36478             |0.8176100000000001  |
|[5, 3]    |10                  |0.9          |0.9     |300   |0.36478             |0.8176100000000001  |
|[5, 3]    |10                  |0.9          |0.9     |500   |0.364773            |0.8176100000000001  |
|[10, 8, 3]|10                  |1e-05        |1e-05   |300   |0.321654            |0.8176100000000001  |
|[10, 8, 3]|10                  |1e-05        |1e-05   |500   |0.32432300000000003 |0.8176100000000001  |
|[10, 8, 3]|10                  |1e-05        |0.001   |50    |0.5506979999999999  |0.8176100000000001  |
|[10, 8, 3]|10                  |1e-05        |0.001   |300   |0.320548            |0.8176100000000001  |
|[10, 8, 3]|10                  |1e-05        |0.001   |500   |0.32160900000000003 |0.8176100000000001  |
|[10, 8, 3]|10                  |1e-05        |0.9     |50    |0.321504            |0.8176100000000001  |
|[10, 8, 3]|10                  |1e-05        |0.9     |300   |0.310711            |0.8176100000000001  |
|[10, 8, 3]|10                  |1e-05        |0.9     |500   |0.31049099999999996 |0.8176100000000001  |
|[10, 8, 3]|10                  |0.01         |1e-05   |50    |0.29630300000000004 |0.8176100000000001  |
|[10, 8, 3]|10                  |0.01         |0.001   |50    |0.302645            |0.8176100000000001  |
|[10, 8, 3]|10                  |0.9          |0.9     |50    |0.36478             |0.8176100000000001  |
|[10, 8, 3]|10                  |0.9          |0.9     |300   |0.364779            |0.8176100000000001  |
|[10, 8, 3]|10                  |0.9          |0.9     |500   |0.36478             |0.8176100000000001  |
|[11, 8, 3]|10                  |1e-05        |1e-05   |300   |0.33395199999999997 |0.8176100000000001  |
|[11, 8, 3]|10                  |1e-05        |1e-05   |500   |0.321036            |0.8176100000000001  |
|[11, 8, 3]|10                  |1e-05        |0.001   |50    |0.441325            |0.8176100000000001  |
|[11, 8, 3]|10                  |1e-05        |0.001   |300   |0.34225900000000004 |0.8176100000000001  |
|[11, 8, 3]|10                  |1e-05        |0.001   |500   |0.31757399999999997 |0.8176100000000001  |
|[11, 8, 3]|10                  |1e-05        |0.9     |50    |0.320513            |0.8176100000000001  |
|[11, 8, 3]|10                  |1e-05        |0.9     |300   |0.310006            |0.8176100000000001  |
|[11, 8, 3]|10                  |1e-05        |0.9     |500   |0.310971            |0.8176100000000001  |
|[11, 8, 3]|10                  |0.01         |1e-05   |50    |0.30082600000000004 |0.8176100000000001  |
|[11, 8, 3]|10                  |0.01         |0.001   |50    |0.253212            |0.8176100000000001  |
|[11, 8, 3]|10                  |0.9          |1e-05   |50    |0.219365            |0.8176100000000001  |
|[11, 8, 3]|10                  |0.9          |0.9     |50    |0.36478             |0.8176100000000001  |
|[11, 8, 3]|10                  |0.9          |0.9     |300   |0.364779            |0.8176100000000001  |
|[11, 8, 3]|10                  |0.9          |0.9     |500   |0.36478             |0.8176100000000001  |
|[10, 8, 3]|70                  |0.01         |0.001   |300   |0.267005            |0.8176939999999999  |
|[5, 3]    |30                  |0.01         |0.9     |300   |0.251217            |0.818372            |
|[11, 8, 3]|30                  |0.9          |0.001   |500   |0.286586            |0.818372            |
|[10, 8, 3]|70                  |0.9          |1e-05   |500   |0.305987            |0.818588            |
|[11, 8, 3]|70                  |0.01         |0.9     |50    |0.27023400000000003 |0.818588            |
|[10, 8, 3]|70                  |0.9          |1e-05   |50    |0.26482             |0.8194819999999999  |
|[11, 8, 3]|70                  |0.9          |0.001   |50    |0.270303            |0.820375            |
|[5, 3]    |70                  |0.01         |0.001   |300   |0.263766            |0.821269            |
|[11, 8, 3]|70                  |0.01         |0.001   |300   |0.260011            |0.821269            |
|[11, 8, 3]|70                  |0.01         |1e-05   |300   |0.26145999999999997 |0.8221629999999999  |
|[10, 8, 3]|30                  |0.9          |0.001   |300   |0.275113            |0.8225469999999999  |
|[5, 3]    |70                  |0.9          |0.001   |300   |0.305444            |0.8230559999999999  |
|[11, 8, 3]|70                  |0.01         |1e-05   |500   |0.261421            |0.8230559999999999  |
|[5, 3]    |10                  |0.9          |0.001   |50    |0.22451500000000002 |0.8238989999999999  |
|[10, 8, 3]|10                  |0.01         |0.9     |300   |0.209166            |0.8238989999999999  |
|[5, 3]    |70                  |0.9          |0.001   |50    |0.264786            |0.82395             |
|[10, 8, 3]|70                  |0.01         |1e-05   |300   |0.261412            |0.82395             |
|[11, 8, 3]|30                  |0.01         |0.9     |50    |0.25766100000000003 |0.824635            |
|[11, 8, 3]|30                  |0.9          |1e-05   |50    |0.268764            |0.824635            |
|[5, 3]    |70                  |0.01         |1e-05   |300   |0.260948            |0.824844            |
|[10, 8, 3]|70                  |0.01         |0.001   |500   |0.266842            |0.824844            |
|[10, 8, 3]|70                  |0.9          |0.001   |50    |0.26606399999999997 |0.824844            |
|[11, 8, 3]|70                  |0.01         |0.001   |500   |0.26614299999999996 |0.824844            |
|[11, 8, 3]|70                  |0.9          |1e-05   |50    |0.26882399999999995 |0.824844            |
|[5, 3]    |70                  |0.01         |1e-05   |500   |0.259581            |0.825737            |
|[5, 3]    |70                  |0.01         |0.001   |500   |0.263731            |0.825737            |
|[10, 8, 3]|70                  |0.01         |1e-05   |500   |0.266409            |0.8266309999999999  |
|[5, 3]    |30                  |1e-05        |1e-05   |300   |0.407723            |0.826722            |
|[5, 3]    |30                  |1e-05        |1e-05   |500   |0.338818            |0.826722            |
|[5, 3]    |30                  |1e-05        |0.001   |500   |0.33119699999999996 |0.826722            |
|[5, 3]    |30                  |1e-05        |0.9     |50    |0.34282199999999996 |0.826722            |
|[5, 3]    |30                  |1e-05        |0.9     |300   |0.296688            |0.826722            |
|[5, 3]    |30                  |1e-05        |0.9     |500   |0.29733000000000004 |0.826722            |
|[5, 3]    |30                  |0.01         |1e-05   |50    |0.279475            |0.826722            |
|[5, 3]    |30                  |0.01         |0.001   |50    |0.279049            |0.826722            |
|[5, 3]    |30                  |0.9          |0.9     |50    |0.346555            |0.826722            |
|[5, 3]    |30                  |0.9          |0.9     |300   |0.34628000000000003 |0.826722            |
|[5, 3]    |30                  |0.9          |0.9     |500   |0.346555            |0.826722            |
|[10, 8, 3]|30                  |1e-05        |1e-05   |50    |0.593845            |0.826722            |
|[10, 8, 3]|30                  |1e-05        |1e-05   |300   |0.328063            |0.826722            |
|[10, 8, 3]|30                  |1e-05        |1e-05   |500   |0.32456599999999997 |0.826722            |
|[10, 8, 3]|30                  |1e-05        |0.001   |50    |0.43736800000000003 |0.826722            |
|[10, 8, 3]|30                  |1e-05        |0.001   |300   |0.346821            |0.826722            |
|[10, 8, 3]|30                  |1e-05        |0.001   |500   |0.325911            |0.826722            |
|[10, 8, 3]|30                  |1e-05        |0.9     |50    |0.309498            |0.826722            |
|[10, 8, 3]|30                  |1e-05        |0.9     |300   |0.298363            |0.826722            |
|[10, 8, 3]|30                  |1e-05        |0.9     |500   |0.29747399999999996 |0.826722            |
|[10, 8, 3]|30                  |0.01         |1e-05   |50    |0.29351700000000003 |0.826722            |
|[10, 8, 3]|30                  |0.01         |0.001   |50    |0.28024699999999997 |0.826722            |
|[10, 8, 3]|30                  |0.9          |1e-05   |50    |0.253629            |0.826722            |
|[10, 8, 3]|30                  |0.9          |0.001   |50    |0.25861             |0.826722            |
|[10, 8, 3]|30                  |0.9          |0.9     |50    |0.3454              |0.826722            |
|[10, 8, 3]|30                  |0.9          |0.9     |300   |0.346554            |0.826722            |
|[10, 8, 3]|30                  |0.9          |0.9     |500   |0.346555            |0.826722            |
|[11, 8, 3]|30                  |1e-05        |1e-05   |300   |0.32355             |0.826722            |
|[11, 8, 3]|30                  |1e-05        |1e-05   |500   |0.34524             |0.826722            |
|[11, 8, 3]|30                  |1e-05        |0.001   |300   |0.331858            |0.826722            |
|[11, 8, 3]|30                  |1e-05        |0.001   |500   |0.314911            |0.826722            |
|[11, 8, 3]|30                  |1e-05        |0.9     |50    |0.313535            |0.826722            |
|[11, 8, 3]|30                  |1e-05        |0.9     |300   |0.29785500000000004 |0.826722            |
|[11, 8, 3]|30                  |1e-05        |0.9     |500   |0.29823             |0.826722            |
|[11, 8, 3]|30                  |0.01         |1e-05   |50    |0.269285            |0.826722            |
|[11, 8, 3]|30                  |0.01         |0.001   |50    |0.267242            |0.826722            |
|[11, 8, 3]|30                  |0.9          |0.9     |50    |0.346555            |0.826722            |
|[11, 8, 3]|30                  |0.9          |0.9     |300   |0.346531            |0.826722            |
|[11, 8, 3]|30                  |0.9          |0.9     |500   |0.346555            |0.826722            |
|[5, 3]    |70                  |0.01         |0.9     |300   |0.263443            |0.827525            |
|[5, 3]    |70                  |0.01         |0.9     |500   |0.275279            |0.827525            |
|[5, 3]    |70                  |0.9          |1e-05   |50    |0.272754            |0.828418            |
|[10, 8, 3]|70                  |0.01         |0.9     |300   |0.276555            |0.828418            |
|[5, 3]    |30                  |0.01         |1e-05   |500   |0.24993400000000002 |0.82881             |
|[10, 8, 3]|30                  |0.01         |1e-05   |500   |0.25381             |0.82881             |
|[11, 8, 3]|30                  |0.01         |1e-05   |300   |0.252566            |0.82881             |
|[5, 3]    |10                  |0.9          |1e-05   |50    |0.211657            |0.830189            |
|[11, 8, 3]|10                  |0.01         |0.9     |300   |0.20863800000000002 |0.830189            |
|[11, 8, 3]|10                  |0.9          |0.001   |50    |0.199096            |0.830189            |
|[5, 3]    |30                  |0.01         |1e-05   |300   |0.252424            |0.8308979999999999  |
|[5, 3]    |30                  |0.9          |0.001   |300   |0.270403            |0.8308979999999999  |
|[10, 8, 3]|70                  |0.01         |0.9     |500   |0.28778899999999996 |0.831099            |
|[5, 3]    |30                  |0.01         |0.9     |500   |0.2528              |0.8329850000000001  |
|[10, 8, 3]|30                  |0.01         |1e-05   |300   |0.25261500000000003 |0.8329850000000001  |
|[10, 8, 3]|30                  |0.01         |0.001   |300   |0.25283             |0.8329850000000001  |
|[10, 8, 3]|30                  |0.01         |0.9     |50    |0.255738            |0.8329850000000001  |
|[11, 8, 3]|30                  |0.01         |0.001   |300   |0.251553            |0.8329850000000001  |
|[5, 3]    |30                  |0.01         |0.001   |300   |0.252411            |0.835073            |
|[5, 3]    |30                  |0.01         |0.001   |500   |0.250815            |0.835073            |
|[5, 3]    |30                  |0.01         |0.9     |50    |0.253209            |0.835073            |
|[10, 8, 3]|30                  |0.9          |0.001   |500   |0.269847            |0.835073            |
|[11, 8, 3]|30                  |0.01         |0.001   |500   |0.250603            |0.835073            |
|[5, 3]    |10                  |0.01         |0.9     |500   |0.203282            |0.836478            |
|[5, 3]    |10                  |0.9          |1e-05   |500   |0.22042899999999999 |0.836478            |
|[5, 3]    |10                  |0.9          |0.001   |500   |0.231582            |0.836478            |
|[10, 8, 3]|10                  |0.9          |1e-05   |50    |0.225212            |0.836478            |
|[10, 8, 3]|10                  |0.9          |0.001   |300   |0.220656            |0.836478            |
|[11, 8, 3]|10                  |0.01         |0.9     |500   |0.206742            |0.836478            |
|[11, 8, 3]|10                  |0.9          |1e-05   |500   |0.24407399999999999 |0.836478            |
|[11, 8, 3]|10                  |0.9          |0.001   |300   |0.231275            |0.836478            |
|[11, 8, 3]|10                  |0.9          |0.001   |500   |0.26364099999999996 |0.836478            |
|[10, 8, 3]|30                  |0.01         |0.001   |500   |0.25258200000000003 |0.837161            |
|[10, 8, 3]|30                  |0.01         |0.9     |500   |0.248863            |0.837161            |
|[11, 8, 3]|30                  |0.01         |1e-05   |500   |0.25295500000000004 |0.8413360000000001  |
|[11, 8, 3]|30                  |0.9          |0.001   |50    |0.26416300000000004 |0.8413360000000001  |
|[5, 3]    |10                  |0.01         |0.9     |50    |0.223498            |0.842767            |
|[11, 8, 3]|10                  |0.01         |0.9     |50    |0.223541            |0.842767            |
|[10, 8, 3]|10                  |0.01         |0.9     |50    |0.22249699999999997 |0.849057            |
|[10, 8, 3]|10                  |0.9          |1e-05   |300   |0.221057            |0.849057            |
|[10, 8, 3]|10                  |0.9          |1e-05   |500   |0.231658            |0.849057            |
|[10, 8, 3]|10                  |0.01         |0.9     |500   |0.202654            |0.8553459999999999  |
|[10, 8, 3]|10                  |0.9          |0.001   |50    |0.226662            |0.8553459999999999  |
|[11, 8, 3]|10                  |0.01         |1e-05   |500   |0.218088            |0.8553459999999999  |
|[11, 8, 3]|10                  |0.9          |1e-05   |300   |0.22502199999999997 |0.8553459999999999  |
|[5, 3]    |10                  |0.01         |1e-05   |300   |0.21853899999999998 |0.861635            |
|[5, 3]    |10                  |0.01         |1e-05   |500   |0.209581            |0.861635            |
|[5, 3]    |10                  |0.01         |0.001   |300   |0.21756199999999998 |0.861635            |
|[10, 8, 3]|10                  |0.01         |0.001   |300   |0.22059             |0.861635            |
|[10, 8, 3]|10                  |0.9          |0.001   |500   |0.18155             |0.861635            |
|[11, 8, 3]|10                  |0.01         |1e-05   |300   |0.21825799999999998 |0.861635            |
|[11, 8, 3]|10                  |0.01         |0.001   |300   |0.220989            |0.861635            |
|[11, 8, 3]|10                  |0.01         |0.001   |500   |0.218086            |0.861635            |
|[5, 3]    |10                  |0.01         |0.001   |500   |0.21806599999999998 |0.867925            |
|[5, 3]    |10                  |0.01         |0.9     |300   |0.204746            |0.867925            |
|[10, 8, 3]|10                  |0.01         |1e-05   |300   |0.220096            |0.867925            |
|[10, 8, 3]|10                  |0.01         |1e-05   |500   |0.21808400000000003 |0.867925            |
|[10, 8, 3]|10                  |0.01         |0.001   |500   |0.217531            |0.867925            |


#### Discução dos resultados

A tabela acima compara diferentes arquiteturas ([5,3], [10,8,3], [11,8,3]) com diferentes valores de taxa de aprendizado(0.00001, 0.01, 0.9), de momentums (0.00001, 0.01, 0.9), de epocas (50, 300, 500) e de porcentagem do conjunto de teste (10, 30, 70).

Algumas combinações ($4\%$) apresentaram resultados muito distoantes, indicando possíveis *outliers*. As demais combinações apresentaram resultados bastante consistentes, variando entre $80\%$ e $86\%$ de acurácia.

Dos piores resultados ($acuracia <= 0.238994$), é notável que eles possuem *learning rate*, *momentum* e número de épocas baixo. Muito provavelmente o que ocorreu então foi o underfitting.

Já nos outros casos, as variações são praticamente irrelevantes, já que todos conseguem acurácias altas. Os modelos com $acuracia >= 0.861635$ tendem a ter muitas épocas e porcentagem de teste baixa, o que pode indicar que isso é favorável para o modelo. No entanto como há pouca diferença entre a performance desse modelo e os outros, isso pode ser apenas acaso.






## Dataset 2: default_features_1059_tracks.txt  (regressão / aproximação)





### Preprocessando Dataset 2

O conjunto contém características sonoras de 1059 canções regionais e as coordenadas geográficas de sua origem. O objetivo do conjunto é permitir treinar uma rede neural que consiga identificar a origem de uma música a partir de suas características.

As primeiras 68 colunas representam características sonoras extraídas de arquivos .WAV com o programa MARSYAS. As duas últimas colunas representam a latitude e longitude de onde a música foi criada, respectivamente.

Os processos de normalização e randomização são similares aos aplicados ao conjunto 1, com a diferença sendo que tanto X quanto Y são normalizados. 

In [4]:
X = []
Y = []

with open('default_features_1059_tracks.txt') as f:
  lines = f.readlines()
  for i in range(len(lines)):
    splits = lines[i].split(',')
    x_i = splits[0:len(splits)-2]# ignora as ultimas 2 
    y_i = splits[len(splits)-2:] # pega as duas ultimas posições onde está o y, replace os espaços e deixar tudo em minusculo para usar o map
    x_i = list( map(float, x_i) ) # convert todos os elementos de x_i para float
    y_i = list( map(float, y_i) ) # convert todos os elementos de x_i para float
    X.append(x_i)
    Y.append(y_i)
    

# normalizando entre 0 e 1 usando o min max
Y_or_min, Y_or_ptp = np.min(Y, axis=0), np.ptp(Y, axis=0)
X = (X - np.min(X, axis=0))/np.ptp(X, axis=0) 
Y = (Y - np.min(Y, axis=0))/np.ptp(Y, axis=0)

XY = list(zip(X,Y))

random.shuffle(XY)

X, Y = zip(*XY)

del XY

X = np.array(X)
Y = np.array(Y)

print(Y.shape)
print("Total de Amostras:", X.shape[0], " Número de Features: ", X.shape[1])

(1059, 2)
Total de Amostras: 1059  Número de Features:  68


### Criando o modelo neural para o problema de regressão

Assim como com o conjunto 1, uma série de testes, variando diversos parâmetros, foi executada, com duas diferenças:
- a função de ativação permaneceu a logística para todas as camadas, exceto a última, que utilizou a função linear;
- ao invés de calcular a acurácia para cada combinação de parâmetros, foi calculado o erro quadrático médio (MSE).

In [0]:
'''#definindo listas para descida coordenada de encosta
topologies_list = [[5,3],[10,3],[10,8,3],[11,8,3],[11,9,3]] # em cada layer cada item da lista indica 1 nova camada e o numero de neuronios é igual ao numero de cada posição ex: [7,4,3] cria 3 camadas sendo a primeira com 7 neuronios, a segunda com 4 e a ultima com 3. 
learning_rates_list= [0.00001,0.0001,0.001,0.01,0.1,0.5,1]
momentums_list= [0.00001,0.0001,0.001,0.01,0.5,0.9]
epochs_list=[10, 50, 300, 1000]
test_percentages = [10,20,30,40,60,80]# porcentagem de amostras de teste'''

#definindo listas para descida coordenada de encosta
topologies_list = [[5,2],[10,8,2],[11,8,2]] # em cada layer cada item da lista indica 1 nova camada e o numero de neuronios é igual ao numero de cada posição ex: [7,4,3] cria 3 camadas sendo a primeira com 7 neuronios, a segunda com 4 e a ultima com 3. 
learning_rates_list= [0.00001,0.01,0.9] 
momentums_list= [0.00001,0.001,0.9]
epochs_list=[50, 300, 500]
test_percentages = [10,30,70]# porcentagem de amostras de teste

results = []
results_history =[]
for l in tqdm(range(len(topologies_list))):
  for lr in learning_rates_list:
    for mo in momentums_list:
      for ep in epochs_list:
        for percent in test_percentages:
          lenx = len(X)
          skip = int(lenx * (float(percent) / 100))
          Xtrain = X[0:lenx-skip]
          Ytrain = Y[0:lenx-skip]
          Xtest = X[lenx-skip:]
          Ytest = Y[lenx-skip:]
          
          layer = topologies_list[l]
          activations= [logistic for _ in range(len(layer)-1)]# camada com ativação logistica
          activations.append(linear)#ultima ativação linear
          mlp = MLP(input_size=68, layers_sizes=layer,  actv_funcs=activations, loss_func=squared_error, metrics=[("MSE", mean_squared_error)])#create MLP
          history = mlp.fit(Xtrain, Ytrain, ep, loss_thresh=0.0001, learning_rate=lr, momentum=mo, reset_momentum=True, return_history=True, verbose=False)# treinando
          evaluate = mlp.evaluate(Xtest, Ytest)#predizer conjunto de teste
          results.append([layer,percent,lr,mo,ep,evaluate[0][1],evaluate[1][1]])#salvando resultados
          results_history.append(history)#salvando historico do treinamento para cada experimento

### Visualização dos Resultados

No trecho de código abaixo os resultados gerados anteriormente são apresentados em forma de tabela. Cada linha representa uma combinação de parâmetros e seus respectivos resultados, ou seja, perda e o erro quadrático médio.

In [0]:
from tabulate import tabulate
newresults = sorted(results, key=lambda k: k[-1]) 
tabela =tabulate(newresults, headers=[ 'Topologia',"Porcentagem de Teste",'Learning_Rate', "Momentum", "Epocas","Loss Média","Erro quadrático médio"])
with open('resultados-dataset2-classificacao-ordenado.txt', 'w') as outputfile:
    outputfile.write(tabela)
print(tabela)

#### Resultados ordenados pelo menor erro

|Topologia |Porcentagem de Teste|Learning_Rate|Momentum|Epocas|Loss Média          |Erro quadrático medio|
|----------|--------------------|-------------|--------|------|--------------------|---------------------|
|[11, 8, 2]|10                  |0.01         |0.9     |500   |0.0613375           |0.0613375            |
|[5, 2]    |10                  |0.01         |0.9     |300   |0.0615326           |0.0615326            |
|[11, 8, 2]|30                  |0.01         |0.9     |300   |0.061915899999999996|0.061915899999999996 |
|[10, 8, 2]|10                  |0.01         |1e-05   |500   |0.06291000000000001 |0.06291000000000001  |
|[5, 2]    |10                  |0.01         |0.9     |500   |0.0630601           |0.0630601            |
|[10, 8, 2]|10                  |0.01         |0.9     |300   |0.0641658           |0.0641658            |
|[10, 8, 2]|10                  |0.01         |0.9     |500   |0.0650337           |0.0650337            |
|[11, 8, 2]|10                  |0.01         |1e-05   |500   |0.06627330000000001 |0.06627330000000001  |
|[10, 8, 2]|10                  |0.01         |1e-05   |300   |0.0664801           |0.0664801            |
|[11, 8, 2]|10                  |0.01         |0.9     |300   |0.06656269999999999 |0.06656269999999999  |
|[11, 8, 2]|10                  |0.01         |0.001   |300   |0.06686160000000001 |0.06686160000000001  |
|[5, 2]    |10                  |0.01         |1e-05   |500   |0.0669054           |0.0669054            |
|[10, 8, 2]|10                  |0.01         |0.001   |500   |0.0674073           |0.0674073            |
|[5, 2]    |10                  |0.01         |0.001   |500   |0.0680894           |0.0680894            |
|[11, 8, 2]|10                  |0.01         |0.001   |500   |0.0684813           |0.0684813            |
|[5, 2]    |10                  |0.01         |1e-05   |300   |0.0689663           |0.0689663            |
|[10, 8, 2]|10                  |0.01         |0.001   |300   |0.06897930000000001 |0.06897930000000001  |
|[11, 8, 2]|10                  |0.01         |1e-05   |300   |0.0697865           |0.0697865            |
|[10, 8, 2]|30                  |0.01         |1e-05   |500   |0.0707423           |0.0707423            |
|[5, 2]    |10                  |0.01         |0.001   |300   |0.07077660000000001 |0.07077660000000001  |
|[5, 2]    |30                  |0.01         |0.9     |500   |0.0715542           |0.0715542            |
|[11, 8, 2]|30                  |0.01         |0.9     |50    |0.0719656           |0.0719656            |
|[10, 8, 2]|30                  |0.01         |0.001   |500   |0.0721877           |0.0721877            |
|[10, 8, 2]|70                  |0.01         |0.001   |300   |0.0727151           |0.0727151            |
|[11, 8, 2]|70                  |0.01         |1e-05   |500   |0.0728489           |0.0728489            |
|[5, 2]    |70                  |0.01         |0.001   |500   |0.0729402           |0.0729402            |
|[5, 2]    |70                  |0.01         |1e-05   |500   |0.0729473           |0.0729473            |
|[10, 8, 2]|30                  |0.01         |0.9     |50    |0.0731446           |0.0731446            |
|[10, 8, 2]|30                  |0.01         |0.001   |300   |0.0741193           |0.0741193            |
|[5, 2]    |10                  |0.01         |0.9     |50    |0.07414730000000001 |0.07414730000000001  |
|[5, 2]    |70                  |0.01         |0.001   |300   |0.07417610000000001 |0.07417610000000001  |
|[11, 8, 2]|70                  |0.01         |0.001   |500   |0.0742416           |0.0742416            |
|[5, 2]    |30                  |0.01         |0.9     |50    |0.0744158           |0.0744158            |
|[11, 8, 2]|10                  |0.01         |1e-05   |50    |0.07464989999999999 |0.07464989999999999  |
|[11, 8, 2]|10                  |0.01         |0.9     |50    |0.07475830000000001 |0.07475830000000001  |
|[11, 8, 2]|30                  |0.01         |1e-05   |300   |0.0750433           |0.0750433            |
|[5, 2]    |30                  |0.01         |0.001   |500   |0.07519830000000001 |0.07519830000000001  |
|[10, 8, 2]|70                  |0.01         |1e-05   |500   |0.07533989999999999 |0.07533989999999999  |
|[11, 8, 2]|70                  |0.01         |0.001   |300   |0.07546660000000001 |0.07546660000000001  |
|[5, 2]    |30                  |0.01         |1e-05   |500   |0.07554169999999999 |0.07554169999999999  |
|[10, 8, 2]|70                  |0.01         |1e-05   |300   |0.0757071           |0.0757071            |
|[11, 8, 2]|30                  |0.01         |0.001   |300   |0.075785            |0.075785             |
|[5, 2]    |70                  |0.01         |1e-05   |300   |0.0758722           |0.0758722            |
|[11, 8, 2]|30                  |0.01         |1e-05   |500   |0.07613810000000001 |0.07613810000000001  |
|[11, 8, 2]|70                  |0.01         |1e-05   |300   |0.0762997           |0.0762997            |
|[11, 8, 2]|70                  |0.01         |0.9     |50    |0.0764172           |0.0764172            |
|[11, 8, 2]|30                  |0.01         |0.001   |500   |0.076463            |0.076463             |
|[10, 8, 2]|70                  |0.01         |0.001   |500   |0.076704            |0.076704             |
|[5, 2]    |30                  |0.01         |1e-05   |300   |0.0768039           |0.0768039            |
|[10, 8, 2]|10                  |0.01         |0.9     |50    |0.0769372           |0.0769372            |
|[10, 8, 2]|30                  |0.01         |1e-05   |300   |0.0771802           |0.0771802            |
|[10, 8, 2]|70                  |0.01         |0.9     |50    |0.0773586           |0.0773586            |
|[11, 8, 2]|10                  |0.01         |0.001   |50    |0.0774627           |0.0774627            |
|[5, 2]    |30                  |0.01         |0.9     |300   |0.0783715           |0.0783715            |
|[11, 8, 2]|70                  |0.01         |0.001   |50    |0.0784985           |0.0784985            |
|[5, 2]    |70                  |0.01         |1e-05   |50    |0.07853489999999999 |0.07853489999999999  |
|[5, 2]    |70                  |0.01         |0.9     |50    |0.07870369999999999 |0.07870369999999999  |
|[5, 2]    |30                  |0.01         |0.001   |300   |0.0790346           |0.0790346            |
|[10, 8, 2]|10                  |0.01         |0.001   |50    |0.0791953           |0.0791953            |
|[5, 2]    |10                  |0.01         |0.001   |50    |0.0796181           |0.0796181            |
|[10, 8, 2]|70                  |0.01         |1e-05   |50    |0.0796823           |0.0796823            |
|[11, 8, 2]|70                  |0.01         |1e-05   |50    |0.0802923           |0.0802923            |
|[10, 8, 2]|70                  |0.01         |0.001   |50    |0.0811194           |0.0811194            |
|[5, 2]    |10                  |0.01         |1e-05   |50    |0.0811199           |0.0811199            |
|[5, 2]    |30                  |0.01         |0.001   |50    |0.081251            |0.081251             |
|[10, 8, 2]|10                  |0.01         |1e-05   |50    |0.0814996           |0.0814996            |
|[10, 8, 2]|30                  |0.01         |1e-05   |50    |0.08282039999999999 |0.08282039999999999  |
|[10, 8, 2]|30                  |0.01         |0.9     |300   |0.0829954           |0.0829954            |
|[11, 8, 2]|30                  |0.01         |1e-05   |50    |0.0831668           |0.0831668            |
|[10, 8, 2]|30                  |1e-05        |0.9     |50    |0.0839011           |0.0839011            |
|[5, 2]    |70                  |0.01         |0.001   |50    |0.08494539999999999 |0.08494539999999999  |
|[5, 2]    |30                  |0.01         |1e-05   |50    |0.08496489999999998 |0.08496489999999998  |
|[11, 8, 2]|30                  |1e-05        |0.9     |500   |0.0862805           |0.0862805            |
|[11, 8, 2]|30                  |0.01         |0.001   |50    |0.08659589999999999 |0.08659589999999999  |
|[10, 8, 2]|30                  |0.01         |0.9     |500   |0.0868514           |0.0868514            |
|[10, 8, 2]|30                  |1e-05        |0.9     |500   |0.0870344           |0.0870344            |
|[5, 2]    |30                  |1e-05        |0.9     |500   |0.087616            |0.087616             |
|[11, 8, 2]|30                  |1e-05        |0.001   |300   |0.0884175           |0.0884175            |
|[11, 8, 2]|70                  |1e-05        |0.001   |300   |0.08885610000000001 |0.08885610000000001  |
|[11, 8, 2]|30                  |1e-05        |0.9     |300   |0.0895204           |0.0895204            |
|[10, 8, 2]|70                  |1e-05        |0.001   |500   |0.0901935           |0.0901935            |
|[10, 8, 2]|70                  |1e-05        |0.9     |50    |0.0902467           |0.0902467            |
|[10, 8, 2]|70                  |1e-05        |0.9     |500   |0.0904029           |0.0904029            |
|[10, 8, 2]|10                  |1e-05        |0.9     |500   |0.0904456           |0.0904456            |
|[10, 8, 2]|10                  |1e-05        |0.001   |300   |0.0909596           |0.0909596            |
|[11, 8, 2]|70                  |1e-05        |0.9     |300   |0.0911052           |0.0911052            |
|[5, 2]    |70                  |1e-05        |0.9     |500   |0.0911643           |0.0911643            |
|[10, 8, 2]|70                  |1e-05        |0.9     |300   |0.09176360000000001 |0.09176360000000001  |
|[11, 8, 2]|70                  |1e-05        |0.9     |500   |0.0917668           |0.0917668            |
|[10, 8, 2]|30                  |1e-05        |0.9     |300   |0.09203480000000001 |0.09203480000000001  |
|[11, 8, 2]|70                  |1e-05        |0.001   |500   |0.09243489999999999 |0.09243489999999999  |
|[5, 2]    |30                  |1e-05        |0.9     |50    |0.0924562           |0.0924562            |
|[5, 2]    |30                  |1e-05        |0.9     |300   |0.0927577           |0.0927577            |
|[10, 8, 2]|10                  |1e-05        |0.001   |500   |0.0928112           |0.0928112            |
|[11, 8, 2]|30                  |1e-05        |0.001   |500   |0.0931725           |0.0931725            |
|[11, 8, 2]|10                  |1e-05        |0.001   |500   |0.0931961           |0.0931961            |
|[10, 8, 2]|70                  |1e-05        |0.001   |300   |0.0932673           |0.0932673            |
|[10, 8, 2]|30                  |1e-05        |1e-05   |500   |0.0935184           |0.0935184            |
|[10, 8, 2]|70                  |1e-05        |1e-05   |500   |0.0936365           |0.0936365            |
|[5, 2]    |10                  |1e-05        |1e-05   |500   |0.09382360000000001 |0.09382360000000001  |
|[5, 2]    |70                  |1e-05        |0.9     |300   |0.0941529           |0.0941529            |
|[5, 2]    |30                  |0.9          |0.9     |500   |0.094229            |0.094229             |
|[5, 2]    |30                  |0.9          |0.9     |50    |0.094229            |0.094229             |
|[5, 2]    |30                  |0.9          |0.9     |300   |0.094229            |0.094229             |
|[10, 8, 2]|30                  |0.9          |0.9     |500   |0.094229            |0.094229             |
|[10, 8, 2]|30                  |0.9          |0.9     |50    |0.094229            |0.094229             |
|[10, 8, 2]|30                  |0.9          |0.9     |300   |0.094229            |0.094229             |
|[11, 8, 2]|30                  |0.9          |0.9     |50    |0.094229            |0.094229             |
|[11, 8, 2]|30                  |0.9          |0.9     |300   |0.094229            |0.094229             |
|[11, 8, 2]|30                  |0.9          |0.9     |500   |0.094229            |0.094229             |
|[10, 8, 2]|30                  |0.01         |0.001   |50    |0.094433            |0.094433             |
|[5, 2]    |10                  |1e-05        |0.9     |300   |0.0945184           |0.0945184            |
|[5, 2]    |10                  |1e-05        |0.9     |500   |0.09496             |0.09496              |
|[5, 2]    |70                  |0.01         |0.9     |300   |0.0949881           |0.0949881            |
|[11, 8, 2]|30                  |1e-05        |1e-05   |500   |0.0950539           |0.0950539            |
|[10, 8, 2]|10                  |1e-05        |1e-05   |300   |0.0954957           |0.0954957            |
|[10, 8, 2]|70                  |0.01         |0.9     |300   |0.0959648           |0.0959648            |
|[10, 8, 2]|10                  |1e-05        |0.9     |300   |0.096745            |0.096745             |
|[11, 8, 2]|10                  |1e-05        |0.001   |300   |0.09742289999999999 |0.09742289999999999  |
|[11, 8, 2]|10                  |1e-05        |0.9     |500   |0.09753400000000001 |0.09753400000000001  |
|[10, 8, 2]|30                  |1e-05        |0.001   |50    |0.09781469999999999 |0.09781469999999999  |
|[11, 8, 2]|10                  |1e-05        |1e-05   |50    |0.0978321           |0.0978321            |
|[11, 8, 2]|70                  |1e-05        |0.9     |50    |0.0979234           |0.0979234            |
|[10, 8, 2]|10                  |1e-05        |1e-05   |50    |0.09810830000000001 |0.09810830000000001  |
|[10, 8, 2]|10                  |1e-05        |0.001   |50    |0.0983171           |0.0983171            |
|[11, 8, 2]|10                  |1e-05        |1e-05   |500   |0.098565            |0.098565             |
|[11, 8, 2]|10                  |1e-05        |0.001   |50    |0.0990406           |0.0990406            |
|[11, 8, 2]|10                  |1e-05        |0.9     |300   |0.0991142           |0.0991142            |
|[11, 8, 2]|30                  |1e-05        |0.9     |50    |0.0991349           |0.0991349            |
|[10, 8, 2]|30                  |1e-05        |0.001   |500   |0.0994093           |0.0994093            |
|[11, 8, 2]|10                  |1e-05        |1e-05   |300   |0.09941810000000001 |0.09941810000000001  |
|[11, 8, 2]|70                  |1e-05        |1e-05   |300   |0.0995265           |0.0995265            |
|[11, 8, 2]|30                  |1e-05        |1e-05   |300   |0.100016            |0.100016             |
|[11, 8, 2]|10                  |1e-05        |0.9     |50    |0.100378            |0.100378             |
|[11, 8, 2]|30                  |0.01         |0.9     |500   |0.100539            |0.100539             |
|[10, 8, 2]|10                  |1e-05        |0.9     |50    |0.102403            |0.102403             |
|[5, 2]    |10                  |1e-05        |1e-05   |300   |0.10288699999999999 |0.10288699999999999  |
|[10, 8, 2]|30                  |1e-05        |0.001   |300   |0.10408800000000001 |0.10408800000000001  |
|[5, 2]    |30                  |1e-05        |1e-05   |300   |0.104111            |0.104111             |
|[11, 8, 2]|70                  |1e-05        |1e-05   |500   |0.104781            |0.104781             |
|[5, 2]    |10                  |1e-05        |0.001   |50    |0.10519500000000001 |0.10519500000000001  |
|[10, 8, 2]|30                  |1e-05        |1e-05   |300   |0.10524000000000001 |0.10524000000000001  |
|[11, 8, 2]|30                  |1e-05        |1e-05   |50    |0.105494            |0.105494             |
|[5, 2]    |10                  |1e-05        |0.001   |500   |0.106072            |0.106072             |
|[11, 8, 2]|70                  |0.01         |0.9     |300   |0.106144            |0.106144             |
|[10, 8, 2]|10                  |1e-05        |1e-05   |500   |0.10638399999999999 |0.10638399999999999  |
|[10, 8, 2]|70                  |1e-05        |1e-05   |300   |0.106429            |0.106429             |
|[5, 2]    |10                  |1e-05        |0.9     |50    |0.10836400000000002 |0.10836400000000002  |
|[5, 2]    |70                  |1e-05        |0.9     |50    |0.108768            |0.108768             |
|[10, 8, 2]|70                  |0.01         |0.9     |500   |0.108953            |0.108953             |
|[5, 2]    |70                  |1e-05        |0.001   |300   |0.11023800000000002 |0.11023800000000002  |
|[5, 2]    |70                  |0.01         |0.9     |500   |0.11208900000000001 |0.11208900000000001  |
|[5, 2]    |30                  |1e-05        |0.001   |300   |0.11211099999999999 |0.11211099999999999  |
|[11, 8, 2]|30                  |1e-05        |0.001   |50    |0.11481300000000001 |0.11481300000000001  |
|[5, 2]    |30                  |1e-05        |1e-05   |50    |0.115125            |0.115125             |
|[5, 2]    |70                  |1e-05        |1e-05   |500   |0.117922            |0.117922             |
|[5, 2]    |10                  |1e-05        |1e-05   |50    |0.118221            |0.118221             |
|[5, 2]    |10                  |1e-05        |0.001   |300   |0.11860899999999999 |0.11860899999999999  |
|[11, 8, 2]|70                  |1e-05        |0.001   |50    |0.120744            |0.120744             |
|[5, 2]    |70                  |1e-05        |0.001   |500   |0.121956            |0.121956             |
|[5, 2]    |30                  |1e-05        |1e-05   |500   |0.124474            |0.124474             |
|[10, 8, 2]|30                  |1e-05        |1e-05   |50    |0.12545499999999998 |0.12545499999999998  |
|[5, 2]    |70                  |1e-05        |1e-05   |300   |0.125913            |0.125913             |
|[11, 8, 2]|70                  |0.01         |0.9     |500   |0.126579            |0.126579             |
|[5, 2]    |30                  |1e-05        |0.001   |50    |0.130324            |0.130324             |
|[5, 2]    |30                  |1e-05        |0.001   |500   |0.135397            |0.135397             |
|[10, 8, 2]|70                  |1e-05        |0.001   |50    |0.223336            |0.223336             |
|[11, 8, 2]|70                  |1e-05        |1e-05   |50    |0.309169            |0.309169             |
|[10, 8, 2]|70                  |1e-05        |1e-05   |50    |0.439287            |0.439287             |
|[11, 8, 2]|70                  |0.9          |0.001   |300   |0.579149            |0.579149             |
|[5, 2]    |70                  |0.9          |0.001   |300   |0.579232            |0.579232             |
|[10, 8, 2]|70                  |0.9          |0.001   |300   |0.579304            |0.579304             |
|[11, 8, 2]|70                  |0.9          |0.001   |500   |0.5793229999999999  |0.5793229999999999   |
|[5, 2]    |70                  |0.9          |0.001   |50    |0.579325            |0.579325             |
|[11, 8, 2]|70                  |0.9          |0.001   |50    |0.579333            |0.579333             |
|[10, 8, 2]|70                  |0.9          |0.001   |50    |0.5794020000000001  |0.5794020000000001   |
|[5, 2]    |70                  |0.9          |0.001   |500   |0.579563            |0.579563             |
|[5, 2]    |70                  |0.9          |1e-05   |300   |0.587856            |0.587856             |
|[11, 8, 2]|70                  |0.9          |1e-05   |300   |0.5882609999999999  |0.5882609999999999   |
|[10, 8, 2]|70                  |0.9          |1e-05   |500   |0.5882609999999999  |0.5882609999999999   |
|[11, 8, 2]|70                  |0.9          |1e-05   |500   |0.588263            |0.588263             |
|[5, 2]    |70                  |0.9          |1e-05   |50    |0.5883079999999999  |0.5883079999999999   |
|[11, 8, 2]|70                  |0.9          |1e-05   |50    |0.588309            |0.588309             |
|[5, 2]    |70                  |0.9          |1e-05   |500   |0.588309            |0.588309             |
|[10, 8, 2]|70                  |0.9          |1e-05   |300   |0.58831             |0.58831              |
|[10, 8, 2]|70                  |0.9          |1e-05   |50    |0.58833             |0.58833              |
|[5, 2]    |70                  |1e-05        |1e-05   |50    |0.671461            |0.671461             |
|[10, 8, 2]|30                  |0.9          |0.001   |300   |0.674824            |0.674824             |
|[10, 8, 2]|70                  |0.9          |0.001   |500   |0.6777350000000001  |0.6777350000000001   |
|[10, 8, 2]|30                  |0.9          |1e-05   |500   |0.6892729999999999  |0.6892729999999999   |
|[5, 2]    |30                  |0.9          |1e-05   |50    |0.68948             |0.68948              |
|[5, 2]    |30                  |0.9          |1e-05   |300   |0.689589            |0.689589             |
|[5, 2]    |30                  |0.9          |1e-05   |500   |0.689612            |0.689612             |
|[10, 8, 2]|30                  |0.9          |1e-05   |300   |0.6896260000000001  |0.6896260000000001   |
|[11, 8, 2]|30                  |0.9          |1e-05   |50    |0.689713            |0.689713             |
|[11, 8, 2]|30                  |0.9          |1e-05   |500   |0.6898340000000001  |0.6898340000000001   |
|[10, 8, 2]|30                  |0.9          |0.001   |50    |0.690157            |0.690157             |
|[5, 2]    |30                  |0.9          |0.001   |500   |0.690195            |0.690195             |
|[10, 8, 2]|30                  |0.9          |0.001   |500   |0.690199            |0.690199             |
|[11, 8, 2]|30                  |0.9          |0.001   |50    |0.6902              |0.6902               |
|[11, 8, 2]|30                  |0.9          |0.001   |500   |0.6902010000000001  |0.6902010000000001   |
|[5, 2]    |30                  |0.9          |0.001   |300   |0.6902020000000001  |0.6902020000000001   |
|[11, 8, 2]|30                  |0.9          |0.001   |300   |0.69026             |0.69026              |
|[11, 8, 2]|30                  |0.9          |1e-05   |300   |0.692088            |0.692088             |
|[5, 2]    |30                  |0.9          |0.001   |50    |0.724489            |0.724489             |
|[10, 8, 2]|30                  |0.9          |1e-05   |50    |0.735938            |0.735938             |
|[5, 2]    |70                  |1e-05        |0.001   |50    |0.806186            |0.806186             |
|[10, 8, 2]|10                  |0.9          |1e-05   |300   |0.871767            |0.871767             |
|[11, 8, 2]|70                  |0.9          |0.9     |500   |1.6486900000000002  |1.6486900000000002   |
|[5, 2]    |70                  |0.9          |0.9     |500   |1.6486900000000002  |1.6486900000000002   |
|[5, 2]    |70                  |0.9          |0.9     |300   |1.6486900000000002  |1.6486900000000002   |
|[10, 8, 2]|70                  |0.9          |0.9     |300   |1.6486900000000002  |1.6486900000000002   |
|[10, 8, 2]|70                  |0.9          |0.9     |500   |1.6486900000000002  |1.6486900000000002   |
|[11, 8, 2]|70                  |0.9          |0.9     |50    |1.6486900000000002  |1.6486900000000002   |
|[11, 8, 2]|70                  |0.9          |0.9     |300   |1.6486900000000002  |1.6486900000000002   |
|[5, 2]    |70                  |0.9          |0.9     |50    |1.6486900000000002  |1.6486900000000002   |
|[10, 8, 2]|70                  |0.9          |0.9     |50    |1.6486900000000002  |1.6486900000000002   |
|[10, 8, 2]|10                  |0.9          |0.001   |500   |2.1024700000000003  |2.1024700000000003   |
|[11, 8, 2]|10                  |0.9          |0.001   |50    |2.1476              |2.1476               |
|[5, 2]    |10                  |0.9          |0.001   |50    |2.1508599999999998  |2.1508599999999998   |
|[5, 2]    |10                  |0.9          |0.001   |300   |2.15147             |2.15147              |
|[10, 8, 2]|10                  |0.9          |0.001   |300   |2.15148             |2.15148              |
|[5, 2]    |10                  |0.9          |0.001   |500   |2.15155             |2.15155              |
|[11, 8, 2]|10                  |0.9          |0.001   |500   |2.15157             |2.15157              |
|[11, 8, 2]|10                  |0.9          |0.001   |300   |2.15163             |2.15163              |
|[10, 8, 2]|10                  |0.9          |0.001   |50    |2.15198             |2.15198              |
|[10, 8, 2]|10                  |0.9          |1e-05   |50    |2.1699200000000003  |2.1699200000000003   |
|[5, 2]    |10                  |0.9          |1e-05   |300   |2.17017             |2.17017              |
|[11, 8, 2]|10                  |0.9          |1e-05   |300   |2.17017             |2.17017              |
|[11, 8, 2]|10                  |0.9          |1e-05   |500   |2.17026             |2.17026              |
|[10, 8, 2]|10                  |0.9          |1e-05   |500   |2.17026             |2.17026              |
|[5, 2]    |10                  |0.9          |1e-05   |500   |2.17027             |2.17027              |
|[5, 2]    |10                  |0.9          |1e-05   |50    |2.17079             |2.17079              |
|[11, 8, 2]|10                  |0.9          |1e-05   |50    |2.18728             |2.18728              |
|[10, 8, 2]|10                  |0.9          |0.9     |500   |2.27618             |2.27618              |
|[5, 2]    |10                  |0.9          |0.9     |50    |2.27618             |2.27618              |
|[5, 2]    |10                  |0.9          |0.9     |300   |2.27618             |2.27618              |
|[10, 8, 2]|10                  |0.9          |0.9     |300   |2.27618             |2.27618              |
|[11, 8, 2]|10                  |0.9          |0.9     |50    |2.27618             |2.27618              |
|[11, 8, 2]|10                  |0.9          |0.9     |300   |2.27618             |2.27618              |
|[11, 8, 2]|10                  |0.9          |0.9     |500   |2.27618             |2.27618              |
|[5, 2]    |10                  |0.9          |0.9     |500   |2.27618             |2.27618              |
|[10, 8, 2]|10                  |0.9          |0.9     |50    |2.27618             |2.27618              |


#### Discução dos resultados

A tabela acima compara diferentes arquiteturas ([5,2], [10,8,2], [11,8,2]) com diferentes valores de taxa de aprendizado(0.00001, 0.01, 0.9), de momentums (0.00001, 0.01, 0.9), de epocas (50, 300, 500) e de porcentagem do conjunto de teste (10, 30, 70).   

Percebe-se na tabela que os melhores resultados, os que possuem um Erro quadrático médio de 0.06, são independente da arquitetura utilizada e usam uma taxa de apendizado de 0.01, o melhor termo momentum varia dependo das circunstâncias, porém os 3 melhores resultados foram alcançados com o termo momentun igual a 0.9. Adicionalmente os melhores resultados são alcançados quando se tem uma porcentagem baixa de amostras para teste (10%,30%). Quando utiliza-se uma taxa de aprendizado alta os modelos não tiveram bom desempenho, isso se deve a ele ter pulado o minimo local/global devido ao tamanho do passo ser muito alto. 

Os piores resultados foram alcançados quando a taxa de aprendizado e o termo momentun foram proximos de 1(0.9) e utilizando apenas 10% das amostras para teste. Acredita-se que o baixo desempenho dos modelos nesse cenário é devido a alta taxa de aprendizado e a caracteristica do termo momentun, que é uma constante que determina o efeito das mudanças passadas dos pesos na direção atual do movimento no espaço de pesos.Um ponto interessante é que em alguns cenários com mais amostras de teste (30%) e menos amostras de treinamento(70%) os modelos com momentun e taxa de aprendizado igual a 0.9 tiveram melhor desempenho do que quando possuem mais amostras de treinamento (90%).

Percebe-se também que os melhores resultados tendem a ter mais épocas, enquanto os piores tendem a ter menos. Isso mostra que um modelo treinado por mais épocas pode vir a ser melhor. Isso não foi realizado nas combinações por questões do custo computacional de treinar por tantas épocas.

##Notas Importantes Sobre a Regressão e END-TO-END

Na regressão os dados foram normalizados tanto na entrada (X) quanto na saída (Y). Isso foi feito para manter a escala perto de 0, que é a região de convergência da função de ativação, e também, para diminuir a chance de ocorrer overflow. Para obter o resultado predito na mesma escala da entrada original, basta fazer o processo de normalização ao contrário. Vou demosntrar a seguir o que precisa ser feito, end-to-end, para esclarecer melhor o que precisa ser feito.

O end-to-end feito abaixo para regressão é idêntico ao feito para a classificação, com a diferença sendo a escala do Y, que não é escalado na classificação, e as funções de métrica e ativação.

Inicialmente, obtem-se o dataset.

In [0]:
X = []
Y = []

with open('default_features_1059_tracks.txt') as f:
  lines = f.readlines()
  for i in range(len(lines)):
    splits = lines[i].split(',')
    x_i = splits[0:len(splits)-2]# ignora as ultimas 2 
    y_i = splits[len(splits)-2:] # pega as duas ultimas posições onde está o y, replace os espaços e deixar tudo em minusculo para usar o map
    x_i = list( map(float, x_i) ) # convert todos os elementos de x_i para float
    y_i = list( map(float, y_i) ) # convert todos os elementos de x_i para float
    X.append(x_i)
    Y.append(y_i)
    

# normalizando entre 0 e 1 usando o min max
Y_or_min, Y_or_ptp = np.min(Y, axis=0), np.ptp(Y, axis=0)
X = (X - np.min(X, axis=0))/np.ptp(X, axis=0) 
Y = (Y - np.min(Y, axis=0))/np.ptp(Y, axis=0)

XY = list(zip(X,Y))

random.shuffle(XY)

X, Y = zip(*XY)

del XY

X = np.array(X)
Y = np.array(Y)

Em seguida treina-se a rede, com os dados escalados. Utilizarei arquiteturas e hiperparâmetros definidos aleatóriamente, já que isso é apenas uma demonstração. Esse valores estão maiores que os dos experimentos propositalmente, pois para essa demostração posso utilizar o tempo necessário para treinar esse modelo.

In [31]:
layer = [10, 5, 2]
percent = 10
lr = 0.02
mo = 0.005
ep = 1000


lenx = len(X)
skip = int(lenx * (float(percent) / 100))
Xtrain = X[0:lenx-skip]
Ytrain = Y[0:lenx-skip]
Xtest = X[lenx-skip:]
Ytest = Y[lenx-skip:]

activations= [logistic for _ in range(len(layer)-1)]
activations.append(linear)
mlp = MLP(input_size=68, layers_sizes=layer,  actv_funcs=activations, loss_func=squared_error, metrics=[("MSE", mean_squared_error)])
_ = mlp.fit(Xtrain, Ytrain, ep, loss_thresh=0.0001, learning_rate=lr, momentum=mo, reset_momentum=True, return_history=True, verbose=True)

----------------------------
	EPOCH 0
Loss: 0.08820613241707255
MSE: 0.08820613241707254
----------------------------
	EPOCH 1
Loss: 0.08560749802817678
MSE: 0.0856074980281768
----------------------------
	EPOCH 2
Loss: 0.08408676899202806
MSE: 0.08408676899202806
----------------------------
	EPOCH 3
Loss: 0.083067850455533
MSE: 0.08306785045553301
----------------------------
	EPOCH 4
Loss: 0.08231775871600112
MSE: 0.08231775871600114
----------------------------
	EPOCH 5
Loss: 0.0817254743722186
MSE: 0.08172547437221861
----------------------------
	EPOCH 6
Loss: 0.08123198977873818
MSE: 0.08123198977873818
----------------------------
	EPOCH 7
Loss: 0.08080341428241952
MSE: 0.08080341428241951
----------------------------
	EPOCH 8
Loss: 0.08041908059945926
MSE: 0.08041908059945925
----------------------------
	EPOCH 9
Loss: 0.08006576737903079
MSE: 0.08006576737903079
----------------------------
	EPOCH 10
Loss: 0.07973469923137441
MSE: 0.07973469923137441
------------------------

Em seguida, é feita a pedição da rede, e os dados são levados para a escala original.

In [0]:
pred = (mlp.predict(Xtest) * Y_or_ptp) + Y_or_min
true = (Ytest * Y_or_ptp) + Y_or_min

Por último, é calculada a metrica na escala original

In [33]:
print("Loss na escala do treino: ", mean_squared_error(mlp.predict(Xtest), Ytest))
print("Loss na escala original: ", mean_squared_error(pred, true))

Loss na escala do treino:  0.0780325711858296
Loss na escala original:  2403.5958195247017
